<h1>Innlesning og bearbeiding</h1>

In [3]:
# Oppg. 1

import polars as pl

df1 = pl.read_csv("PKK-2024-kvartal3/PKK-2024-kvartal3.csv", encoding="latin-1")
df2 = pl.read_csv("PKK-2024-kvartal4/PKK-2024-kvartal4.csv", encoding="latin-1")



df = pl.concat([df1, df2])

print(df.columns)

['Kontrollinstruksversjon', 'Første gang registrert', 'Første gang registrert i Norge', 'Kjøretøymerke', 'Kjøretøy Modell', 'Kjøretøy Merkekode', 'Kjøretøy Gruppekode', 'Kjøretøy Gruppeavgift', 'Kjøretøy Tekniskgruppe', 'Drivstofftype', 'Kilometerstand', 'Tillatt totalvekt opp til og med 3500', 'Tillatt totalvekt 3501-7500', 'Tillatt totalvekt over 7500', 'PKK Kontrolltype', 'PKK Kontrollmåned', 'PKK Intervall', 'PKK Frist', 'Trafikkfarlig feil', 'Godkjent', 'Kontrollorganets fylke', 'Kontrollorganets fylkesnummer', 'Ant 1er merknad', 'Ant 2er merknad', 'Ant 3er merknad', 'Ant 4er merknad', 'Ant 2-3er kap 0', 'Ant 2-3er kap 1', 'Ant 2-3er kap 2', 'Ant 2-3er kap 3', 'Ant 2-3er kap 4', 'Ant 2-3er kap 5', 'Ant 2-3er kap 6', 'Ant 2-3er kap 7', 'Ant 2-3er kap 8', 'Ant 2-3er kap 9', 'Ant 2-3er kap 10', 'Ant 00_01', 'Ant 00_02', 'Ant 01_01_01', 'Ant 01_01_02', 'Ant 01_01_03', 'Ant 01_01_04', 'Ant 01_01_05', 'Ant 01_01_06', 'Ant 01_01_07', 'Ant 01_01_08', 'Ant 01_01_09', 'Ant 01_01_10', 'Ant 0

In [4]:
# Oppg. 2

df = df.with_columns(
    pl.col("Godkjent").eq(pl.lit("Ja")).alias("Godkjent (bool)"),
    pl.col("Trafikkfarlig feil").eq(pl.lit("Ja")).alias("Trafikkfarlig feil (bool)")
)

df.drop(["Godkjent", "Trafikkfarlig feil"])

print(df[["Godkjent (bool)", "Trafikkfarlig feil (bool)"]])

shape: (993_661, 2)
┌─────────────────┬───────────────────────────┐
│ Godkjent (bool) ┆ Trafikkfarlig feil (bool) │
│ ---             ┆ ---                       │
│ bool            ┆ bool                      │
╞═════════════════╪═══════════════════════════╡
│ true            ┆ false                     │
│ true            ┆ false                     │
│ true            ┆ false                     │
│ true            ┆ false                     │
│ true            ┆ false                     │
│ …               ┆ …                         │
│ true            ┆ false                     │
│ true            ┆ false                     │
│ false           ┆ false                     │
│ true            ┆ false                     │
│ true            ┆ false                     │
└─────────────────┴───────────────────────────┘


In [5]:
# Oppg. 3



df = df.with_columns(
    pl.col("PKK Kontrollmåned").cast(str).str.slice(0, 4).cast(pl.UInt16).alias("PKK Kontrollår"),
    pl.col("PKK Kontrollmåned").cast(str).str.slice(4, 2).cast(pl.UInt8, strict=False).alias("PKK Kontrollmåned"), #her hadde jeg ugyldige verdier og strict=False gjør at disse verdiene blir NULL
    pl.col("PKK Frist").cast(str).str.slice(0, 4).cast(pl.UInt16).alias("PKK Fristår"),
    pl.col("PKK Frist").cast(str).str.slice(5, 2).cast(pl.UInt8).alias("PKK Fristmåned")

)

df.drop(["PKK Frist"])

print (df[["PKK Kontrollår", "PKK Kontrollmåned", "PKK Fristår", "PKK Fristmåned"]])

shape: (993_661, 4)
┌────────────────┬───────────────────┬─────────────┬────────────────┐
│ PKK Kontrollår ┆ PKK Kontrollmåned ┆ PKK Fristår ┆ PKK Fristmåned │
│ ---            ┆ ---               ┆ ---         ┆ ---            │
│ u16            ┆ u8                ┆ u16         ┆ u8             │
╞════════════════╪═══════════════════╪═════════════╪════════════════╡
│ 2024           ┆ 7                 ┆ 2024        ┆ 7              │
│ 2024           ┆ 7                 ┆ 2024        ┆ 6              │
│ 2024           ┆ 7                 ┆ 2024        ┆ 8              │
│ 2024           ┆ 7                 ┆ 2024        ┆ 8              │
│ 2024           ┆ 7                 ┆ 2024        ┆ 7              │
│ …              ┆ …                 ┆ …           ┆ …              │
│ 2024           ┆ 12                ┆ 2025        ┆ 1              │
│ 2024           ┆ 12                ┆ 2024        ┆ 1              │
│ 2024           ┆ 12                ┆ 2025        ┆ 2              │


In [6]:
# Oppg. 4 

df = df.with_columns(
    pl.when(pl.col("Tillatt totalvekt opp til og med 3500") != "").then(pl.lit("Opp til og med 3500"))
    .when(pl.col("Tillatt totalvekt 3501-7500") != "").then(pl.lit("3501-7500")) 
    .when(pl.col("Tillatt totalvekt over 7500") != "").then(pl.lit("over 7500"))
    .otherwise(pl.lit("NULL"))
    .alias("Tillatt totalvekt")
)

df.drop(["Tillatt totalvekt opp til og med 3500", "Tillatt totalvekt 3501-7500", "Tillatt totalvekt over 7500"])
print(df)

shape: (993_661, 209)
┌─────────────────────────┬────────────────────────┬──────────────────────────┬───────────────┬───┬────────────────┬─────────────┬────────────────┬─────────────────────┐
│ Kontrollinstruksversjon ┆ Første gang registrert ┆ Første gang registrert i ┆ Kjøretøymerke ┆ … ┆ PKK Kontrollår ┆ PKK Fristår ┆ PKK Fristmåned ┆ Tillatt totalvekt   │
│ ---                     ┆ ---                    ┆ Norge                    ┆ ---           ┆   ┆ ---            ┆ ---         ┆ ---            ┆ ---                 │
│ i64                     ┆ i64                    ┆ ---                      ┆ str           ┆   ┆ u16            ┆ u16         ┆ u8             ┆ str                 │
│                         ┆                        ┆ i64                      ┆               ┆   ┆                ┆             ┆                ┆                     │
╞═════════════════════════╪════════════════════════╪══════════════════════════╪═══════════════╪═══╪════════════════╪════════════

<h1>Serialisering og de-serialisering</h1>

In [7]:
# Oppg. 1
%time

df.write_csv("PKK_2024_Q3-4.csv")

CPU times: total: 0 ns
Wall time: 18.1 μs


In [8]:
# Oppg. 2
%time

df.write_parquet("PKK_2024_Q3-4.parquet", compression = 'uncompressed')

CPU times: total: 0 ns
Wall time: 13.6 μs


In [9]:
%time

df.write_parquet("PKK_2024_Q3-4_komprimert.parquet", compression = 'zstd')

CPU times: total: 0 ns
Wall time: 9.3 μs


In [40]:
# Oppg. 3

"""
Wall time CSV:  160 μs
Wall time ukomprimert Parquet: 24.3 μs
Wall time komprimert Parquet: 12.2 μs

Størrelse på CSV: 489 MB
Størrelse på ukomprimert Parquet-fil: 20.9 MB 
Størrelse på komprimert Parquet-fil: 12.1 MB

Fordelene med Parquet er at de er mindre filer og skrives raskere, og en komprimert Parquet-fil sparer spesielt mye tid og plass, 
men CSV-filer kan leses i en vanlig teksteditor, noe Parquet-filer ikke kan.
"""

'\nÅ skrive en CSV-fil tar over dobbelt så lang tid som en ukomprimert Parquet-fil. En komprimert Parquet-fil sparer svært mye tid.\nStørrelse på CSV: 497 MB\nStørrelse på ukomprimert Parquet-fil: 12.2 MB \n'

In [10]:
# Oppg. 4

%time
df_c = pl.read_csv("PKK_2024_Q3-4.csv")

CPU times: total: 0 ns
Wall time: 18.4 μs


In [11]:
%time

df_k = pl.read_parquet("PKK_2024_Q3-4_komprimert.parquet")

print(df_k.columns)

CPU times: total: 0 ns
Wall time: 9.78 μs
['Kontrollinstruksversjon', 'Første gang registrert', 'Første gang registrert i Norge', 'Kjøretøymerke', 'Kjøretøy Modell', 'Kjøretøy Merkekode', 'Kjøretøy Gruppekode', 'Kjøretøy Gruppeavgift', 'Kjøretøy Tekniskgruppe', 'Drivstofftype', 'Kilometerstand', 'Tillatt totalvekt opp til og med 3500', 'Tillatt totalvekt 3501-7500', 'Tillatt totalvekt over 7500', 'PKK Kontrolltype', 'PKK Kontrollmåned', 'PKK Intervall', 'PKK Frist', 'Trafikkfarlig feil', 'Godkjent', 'Kontrollorganets fylke', 'Kontrollorganets fylkesnummer', 'Ant 1er merknad', 'Ant 2er merknad', 'Ant 3er merknad', 'Ant 4er merknad', 'Ant 2-3er kap 0', 'Ant 2-3er kap 1', 'Ant 2-3er kap 2', 'Ant 2-3er kap 3', 'Ant 2-3er kap 4', 'Ant 2-3er kap 5', 'Ant 2-3er kap 6', 'Ant 2-3er kap 7', 'Ant 2-3er kap 8', 'Ant 2-3er kap 9', 'Ant 2-3er kap 10', 'Ant 00_01', 'Ant 00_02', 'Ant 01_01_01', 'Ant 01_01_02', 'Ant 01_01_03', 'Ant 01_01_04', 'Ant 01_01_05', 'Ant 01_01_06', 'Ant 01_01_07', 'Ant 01_01_0

In [ ]:
"""
Wall time for CSV-fil:  70.1 μs
Wall time for komprimert Parquet-fil:  16.5 μs
"""


In [12]:
# Oppg. 5

%time

lf = pl.scan_csv("PKK_2024_Q3-4.csv")
utvalg = lf.select(
    "Kjøretøymerke",
    "Kjøretøy Modell",
    "PKK Kontrollår",
    "Godkjent"
).filter(
    pl.col("Godkjent").is_in(["Nei"])
).collect()


print(utvalg)


CPU times: total: 0 ns
Wall time: 14.1 μs
shape: (348_010, 4)
┌─────────────────┬─────────────────┬────────────────┬──────────┐
│ Kjøretøymerke   ┆ Kjøretøy Modell ┆ PKK Kontrollår ┆ Godkjent │
│ ---             ┆ ---             ┆ ---            ┆ ---      │
│ str             ┆ str             ┆ i64            ┆ str      │
╞═════════════════╪═════════════════╪════════════════╪══════════╡
│ MASSEY-FERGUSON ┆ 7726            ┆ 2024           ┆ Nei      │
│ VOLKSWAGEN      ┆ CADDY           ┆ 2024           ┆ Nei      │
│ CITROEN         ┆ C4 CACTUS       ┆ 2024           ┆ Nei      │
│ FIAT            ┆ 500E            ┆ 2024           ┆ Nei      │
│ PEUGEOT         ┆ PARTNER         ┆ 2024           ┆ Nei      │
│ …               ┆ …               ┆ …              ┆ …        │
│ SKODA           ┆ OCTAVIA         ┆ 2024           ┆ Nei      │
│ TOYOTA          ┆ TOYOTA COROLLA  ┆ 2024           ┆ Nei      │
│ PEUGEOT         ┆ 3008            ┆ 2024           ┆ Nei      │
│ TOYOTA      

In [13]:
%time

lf = pl.scan_parquet("PKK_2024_Q3-4_komprimert.parquet")
utvalg = lf.select(
    "Kjøretøymerke",
    "Kjøretøy Modell",
    "PKK Kontrollår",
    "Godkjent"
).filter(
    pl.col("Godkjent").is_in(["Nei"])
).collect()


print(utvalg)

CPU times: total: 0 ns
Wall time: 15.3 μs
shape: (348_010, 4)
┌─────────────────┬─────────────────┬────────────────┬──────────┐
│ Kjøretøymerke   ┆ Kjøretøy Modell ┆ PKK Kontrollår ┆ Godkjent │
│ ---             ┆ ---             ┆ ---            ┆ ---      │
│ str             ┆ str             ┆ u16            ┆ str      │
╞═════════════════╪═════════════════╪════════════════╪══════════╡
│ MASSEY-FERGUSON ┆ 7726            ┆ 2024           ┆ Nei      │
│ VOLKSWAGEN      ┆ CADDY           ┆ 2024           ┆ Nei      │
│ CITROEN         ┆ C4 CACTUS       ┆ 2024           ┆ Nei      │
│ FIAT            ┆ 500E            ┆ 2024           ┆ Nei      │
│ PEUGEOT         ┆ PARTNER         ┆ 2024           ┆ Nei      │
│ …               ┆ …               ┆ …              ┆ …        │
│ SKODA           ┆ OCTAVIA         ┆ 2024           ┆ Nei      │
│ TOYOTA          ┆ TOYOTA COROLLA  ┆ 2024           ┆ Nei      │
│ PEUGEOT         ┆ 3008            ┆ 2024           ┆ Nei      │
│ TOYOTA      

In [ ]:
"""
Wall time for skanning av CSV: 91.8 μs
Wall time for skanning av komprimert Parquet: 14.3 μs
Siden Parquet er kolonnebasert og CSV er radbasert, så egner Parquet seg spesielt godt til lesing av kolonner.
"""

<h1>Analyse</h1>

In [14]:
# Oppg. 1

unikeBilmerker = df_k.unique(subset=["Kjøretøymerke"], maintain_order=True)

print(unikeBilmerker["Kjøretøymerke"])

shape: (818,)
Series: 'Kjøretøymerke' [str]
[
	"FORD"
	"VOLKSWAGEN"
	"TOYOTA"
	"VOLVO"
	"SKODA"
	…
	"Nova "
	"GS MEPPEL"
	"WBL-HOFFMANN"
	"AUTO BJØRN"
	"ACCURA"
]


In [15]:
# Oppg. 2

 
flest_EU = (
    df_k.group_by("Kjøretøymerke") 
    .agg(pl.col("Godkjent").count().alias("Antall EU-kontroller")) 
    .sort("Antall EU-kontroller", descending=True) 
    .head(3) 
)
print(flest_EU)

shape: (3, 2)
┌───────────────┬──────────────────────┐
│ Kjøretøymerke ┆ Antall EU-kontroller │
│ ---           ┆ ---                  │
│ str           ┆ u32                  │
╞═══════════════╪══════════════════════╡
│ VOLKSWAGEN    ┆ 149575               │
│ TOYOTA        ┆ 117573               │
│ VOLVO         ┆ 79764                │
└───────────────┴──────────────────────┘


In [16]:
# Oppg. 3 

elektrisk_persbil = (
    df_k.filter(
        (pl.col("Kjøretøy Gruppeavgift") == "PERSONBIL") & 
        (pl.col("Drivstofftype") == "Elektrisk")
    )
    .select(["Kjøretøy Gruppeavgift", "Drivstofftype", "Kjøretøymerke", "Kjøretøy Modell"])
    .unique(subset=["Kjøretøymerke", "Kjøretøy Modell"], keep="first", maintain_order=True) 
    .sort("Kjøretøymerke")
)

print(elektrisk_persbil["Kjøretøymerke", "Kjøretøy Modell", "Kjøretøy Gruppeavgift", "Drivstofftype"]) #tar med gruppeavgift og drivstofftype for å sjekke at filteret funket.

shape: (264, 4)
┌───────────────┬─────────────────────┬───────────────────────┬───────────────┐
│ Kjøretøymerke ┆ Kjøretøy Modell     ┆ Kjøretøy Gruppeavgift ┆ Drivstofftype │
│ ---           ┆ ---                 ┆ ---                   ┆ ---           │
│ str           ┆ str                 ┆ str                   ┆ str           │
╞═══════════════╪═════════════════════╪═══════════════════════╪═══════════════╡
│ AUDI          ┆ e-tron Sportback 55 ┆ PERSONBIL             ┆ Elektrisk     │
│ AUDI          ┆ e-tron Sportback 50 ┆ PERSONBIL             ┆ Elektrisk     │
│ AUDI          ┆ e-tron 50           ┆ PERSONBIL             ┆ Elektrisk     │
│ AUDI          ┆ e-tron 55           ┆ PERSONBIL             ┆ Elektrisk     │
│ AUDI          ┆ Audi e-tron         ┆ PERSONBIL             ┆ Elektrisk     │
│ …             ┆ …                   ┆ …                     ┆ …             │
│ XPENG         ┆ G3                  ┆ PERSONBIL             ┆ Elektrisk     │
│ XPENG         ┆ P7    

In [17]:
# Oppg. 4

kontrollAlder = df_k.with_columns(
    (pl.col("PKK Kontrollår") - pl.col("Første gang registrert"))
    .alias("Alder ved kontroll")
)

print(kontrollAlder["Alder ved kontroll"])

shape: (993_661,)
Series: 'Alder ved kontroll' [i64]
[
	18
	20
	12
	14
	34
	…
	19
	14
	6
	1
	19
]


In [18]:
# Oppg. 5

df_andel = (
    df_k.filter(pl.col("PKK Kontrolltype") == "Periodisk")
    .group_by(["Kjøretøymerke", kontrollAlder["Alder ved kontroll"]])
    .agg(
        ((pl.col("Godkjent") == "Ja").sum() / pl.col("Godkjent").count() * 100)
        .round(2)
        .alias("Andel godkjente i prosent")
    )
    .sort("Andel godkjente i prosent", descending=True)
)
print(df_andel)

shape: (7_392, 3)
┌─────────────────┬────────────────────┬───────────────────────────┐
│ Kjøretøymerke   ┆ Alder ved kontroll ┆ Andel godkjente i prosent │
│ ---             ┆ ---                ┆ ---                       │
│ str             ┆ i64                ┆ f64                       │
╞═════════════════╪════════════════════╪═══════════════════════════╡
│ KARMANN MOBIL   ┆ 30                 ┆ 100.0                     │
│ ROLLS-ROYCE     ┆ 3                  ┆ 100.0                     │
│ TESLA MOTORS    ┆ 42                 ┆ 100.0                     │
│ MAN             ┆ 48                 ┆ 100.0                     │
│ HYMER           ┆ 51                 ┆ 100.0                     │
│ …               ┆ …                  ┆ …                         │
│ LAG             ┆ 30                 ┆ 0.0                       │
│ MASSEY-FERGUSON ┆ 1                  ┆ 0.0                       │
│ OVOX            ┆ 16                 ┆ 0.0                       │
│ VBI           

In [26]:
# Oppg. 6

topp3 = (
    df_k.filter(pl.col("PKK Kontrolltype") == "Periodisk") 
    .group_by("Kjøretøymerke").count()
    .sort("count", descending=True)
    .head(3)
    
)

df_andel = df_andel.filter(pl.col("Kjøretøymerke").is_in(topp3["Kjøretøymerke"])).head(3) 
#måtte sette .head(3) her også for å kun få de tre øverste, så da er det kanskje poengløst å sette head(3) i topp3.

print(df_andel)


C:\Users\trudy\AppData\Local\Temp\ipykernel_4744\1115215867.py:5: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  .group_by("Kjøretøymerke").count()


shape: (3, 3)
┌───────────────┬────────────────────┬───────────────────────────┐
│ Kjøretøymerke ┆ Alder ved kontroll ┆ Andel godkjente i prosent │
│ ---           ┆ ---                ┆ ---                       │
│ str           ┆ i64                ┆ f64                       │
╞═══════════════╪════════════════════╪═══════════════════════════╡
│ VOLVO         ┆ 63                 ┆ 100.0                     │
│ VOLVO         ┆ 49                 ┆ 84.62                     │
│ TOYOTA        ┆ 61                 ┆ 80.0                      │
└───────────────┴────────────────────┴───────────────────────────┘


In [30]:
import altair as alt

df_andel.plot.point(

    x=alt.X("Alder ved kontroll", title="Alder ved kontroll"), y="Andel godkjente i prosent", color="Kjøretøymerke", shape="Kjøretøymerke"

)  

# Usikker på om jg har gjort det riktig. Utfra dette ville jeg vel valgt Volvo...?

alt.Chart(...)